In [ ]:
%%writefile app.py
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential

# Load data
with open('intents.json') as file:
    data = json.load(file)

# Prepare data
patterns = []
tags = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

# Encode tags
encoder = LabelEncoder()
encoded_tags = encoder.fit_transform(tags)


Writing app.py


In [ ]:
# Tokenize text data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>")
tokenizer.fit_on_texts(patterns)
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = pad_sequences(sequences, padding='post')

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1
input_length = padded_sequences.shape[1]


In [ ]:
# Save tokenizer and encoder for later use
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(encoder, handle)

In [ ]:
model = Sequential([
    Embedding(vocab_size, 16, input_length=input_length),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(len(set(tags)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
history=model.fit(padded_sequences, np.array(encoded_tags), epochs=200)

Epoch 1/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7782 - loss: 4.4370
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7022 - loss: 2.3113
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6861 - loss: 1.2304
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7678 - loss: 1.0566
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7653 - loss: 0.8320
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7963 - loss: 0.6750 
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8500 - loss: 0.5539
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8511 - loss: 0.4833
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8784 - loss: 0.4462
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8829 - loss: 0.3983
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8968 - loss: 0.3953
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy

In [ ]:
def chatbot_response(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=input_length, padding='post')
    prediction = model.predict(padded)
    tag = encoder.inverse_transform([np.argmax(prediction)])

    for intent in data['intents']:
        if intent['tag'] == tag:
            return np.random.choice(intent['responses'])

In [ ]:
# Initialize Flask app
from flask import Flask, render_template, request, jsonify
import pickle
app = Flask(__name__)

# Route for home page with embedded CSS for styling
@app.route("/")
def home():
    return '''
    <html>
    <head>
        <style>
            /* Basic styling for chatbot */
            body { font-family: Arial, sans-serif; display: flex; justify-content: center; align-items: center; height: 100vh; margin: 0; }
            .chat-container { position: fixed; bottom: 20px; right: 20px; width: 300px; max-height: 400px; border: 1px solid #ccc; border-radius: 8px; overflow: hidden; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2); }
            .chat-header { background-color: #007bff; color: white; padding: 10px; text-align: center; font-weight: bold; }
            .chat-messages { padding: 10px; height: 300px; overflow-y: auto; }
            .chat-bubble { margin-bottom: 10px; padding: 10px; border-radius: 5px; }
            .user-message { background-color: #dcf8c6; text-align: right; }
            .bot-message { background-color: #f1f0f0; text-align: left; }
            .input-container { display: flex; border-top: 1px solid #ddd; }
            .input-container input { width: 100%; padding: 10px; border: none; border-radius: 0; }
            .input-container button { padding: 10px; background-color: #007bff; color: white; border: none; cursor: pointer; }
        </style>
    </head>
    <body>
        <div class="chat-container">
            <div class="chat-header">Virtus Chatbot</div>
            <div class="chat-messages" id="chat-messages"></div>
            <div class="input-container">
                <input type="text" id="user_input" placeholder="Write a message..." autocomplete="off">
                <button onclick="sendMessage()">Send</button>
            </div>
        </div>
        <script>
            // Function to display messages in the chat window
            function displayMessage(text, className) {
                const chatMessages = document.getElementById("chat-messages");
                const messageDiv = document.createElement("div");
                messageDiv.className = "chat-bubble " + className;
                messageDiv.innerText = text;
                chatMessages.appendChild(messageDiv);
                chatMessages.scrollTop = chatMessages.scrollHeight;
            }

            // Function to send a message to the bot
            async function sendMessage() {
                const userInput = document.getElementById("user_input").value;
                if (userInput.trim() === "") return;

                displayMessage(userInput, "user-message");
                document.getElementById("user_input").value = "";

                // Fetch the bot's response
                const response = await fetch("/get_response", {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ "user_input": userInput })
                });
                const data = await response.json();
                displayMessage(data.response, "bot-message");
            }
        </script>
    </body>
    </html>
    '''



In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Set up and run Flask app in the background
!python3 app.py &

# Connect to ngrok to get a public URL
ngrok.set_auth_token("insert_ngrok_auth_token_here")
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


2024-11-12 08:33:50.872502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 08:33:50.903969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 08:33:50.912118: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 08:33:52.164574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Public URL: NgrokTunnel: "https://1ca3-34-90-89-132.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
@app.route("/get_response", methods=["POST"])
def get_response():
    user_input = request.json.get("user_input")
    response = chatbot_response(user_input)
    return jsonify({"response": response})

# Run Flask app
if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 08:34:10] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 08:34:11] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 08:34:16] "POST /get_response HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


...

INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 08:35:59] "POST /get_response HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 08:36:04] "POST /get_response HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 08:36:11] "POST /get_response HTTP/1.1" 200 -




---

